# Part 2 - LangChain Essentials: Building Blocks of AI Applications

Practice the core building blocks: chat models, prompting, structured outputs, and chaining.

## 📦 Setup

In [ ]:
!pip install -q langchain langchain-openai pydantic python-dotenv

In [ ]:
import os
import getpass

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

print("✅ Ready!")

## 🤖 Chat Models

### Basic Chat Model

In [ ]:
from langchain_openai import ChatOpenAI

# Create chat model
chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

response = chat.invoke("Explain quantum physics in simple terms")
print(response.content)

### Testing Different Temperatures

In [ ]:
# Temperature = 0 (deterministic)
chat_precise = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
print("Temperature 0:")
print(chat_precise.invoke("Write a poem about AI").content)
print("\n" + "="*50 + "\n")

# Temperature = 1.5 (creative)
chat_creative = ChatOpenAI(model="gpt-3.5-turbo", temperature=1.5)
print("Temperature 1.5:")
print(chat_creative.invoke("Write a poem about AI").content)

## 💬 Prompting Patterns

### Pattern 1: Role-Based Prompting

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful teacher explaining to a 10-year-old."),
    ("user", "{question}")
])

chain = prompt | chat
response = chain.invoke({"question": "What is DNA?"})
print(response.content)

### Pattern 2: Few-Shot Prompting

In [ ]:
few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a sentiment analyzer."),
    ("user", "Review: This movie was amazing! → Sentiment: Positive"),
    ("user", "Review: Terrible waste of time → Sentiment: Negative"),
    ("user", "Review: {review} → Sentiment:")
])

chain = few_shot_prompt | chat
result = chain.invoke({"review": "It was okay, nothing special"})
print(result.content)

## 📊 Structured Outputs

### Using Pydantic for Structure

In [ ]:
from pydantic import BaseModel, Field

class Person(BaseModel):
    name: str = Field(description="Person's full name")
    age: int = Field(description="Person's age")
    occupation: str = Field(description="Person's job")

# Create structured LLM
structured_llm = chat.with_structured_output(Person)

# Extract structured data
text = "John Smith is a 35-year-old software engineer"
person = structured_llm.invoke(f"Extract person info from: {text}")

print(f"Name: {person.name}")
print(f"Age: {person.age}")
print(f"Occupation: {person.occupation}")

## 🔗 Chaining with LCEL

### Simple Chain

In [ ]:
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
model = ChatOpenAI()
parser = StrOutputParser()

# Chain them together
chain = prompt | model | parser

result = chain.invoke({"topic": "programming"})
print(result)

### Multi-Step Chain

In [ ]:
# Step 1: Generate poem
poem_prompt = ChatPromptTemplate.from_template("Write a 2-line poem about {topic}")

# Step 2: Translate
translate_prompt = ChatPromptTemplate.from_template("Translate to Spanish: {poem}")

# Combine
chain = (
    {"poem": poem_prompt | model | StrOutputParser()}
    | translate_prompt
    | model
    | StrOutputParser()
)

result = chain.invoke({"topic": "ocean"})
print(result)

## 🧪 Practice: Complete Example

In [ ]:
from langchain_core.runnables import RunnableParallel

# Define multiple analysis chains
summary_prompt = ChatPromptTemplate.from_template("Summarize in one sentence: {text}")
sentiment_prompt = ChatPromptTemplate.from_template("What's the sentiment (positive/negative/neutral): {text}")
keywords_prompt = ChatPromptTemplate.from_template("Extract 3 keywords from: {text}")

# Run in parallel
parallel_chain = RunnableParallel(
    summary=summary_prompt | chat | StrOutputParser(),
    sentiment=sentiment_prompt | chat | StrOutputParser(),
    keywords=keywords_prompt | chat | StrOutputParser()
)

text = """AI is transforming technology. Machine learning enables computers to learn 
from data without explicit programming. The future looks exciting!"""

result = parallel_chain.invoke({"text": text})
print("Summary:", result["summary"])
print("Sentiment:", result["sentiment"])
print("Keywords:", result["keywords"])

## 📚 Summary

You've learned:
- Chat models and temperature control
- Prompting patterns (role-based, few-shot)
- Structured outputs with Pydantic
- Chaining with LCEL

**Next:** Part 3 - Tool Calling